# Часть 2. Файнтюнинг диффузионных моделей <br> (2 балла)

Вам предстоит взять обученную text-to-image диффузионную модель [Stable Diffusion 1.5](https://huggingface.co/sd-legacy/stable-diffusion-v1-5), дообучить её в Rectified Flow (RF) режиме по мотивам [Stable Diffusion 3](https://arxiv.org/pdf/2403.03206) и привести результаты с помощью любого солвера и расписания, реализованного в первой части задания.

In [ ]:
!pip install -U diffusers --upgrade
!git clone https://github.com/quickjkee/ysda_hw1

In [ ]:
import csv
import os
import torch
import numpy as np
import functools

from PIL import Image
from tqdm.auto import tqdm

from diffusers import StableDiffusionPipeline, DDIMScheduler
from peft import LoraConfig, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict

%matplotlib inline
import matplotlib.pyplot as plt

torch.set_num_threads(16)

### Utils
Класс датасета и доп. функции, которые потребуются позже. Можно сюда не смотреть

In [ ]:
#---------------------    
# Visualization utils 
#---------------------

def visualize_images(images):
    assert len(images) == 4
    plt.figure(figsize=(12, 3))
    for i, image in enumerate(images):
        plt.subplot(1, 4, i+1)
        plt.imshow(image)
        plt.axis('off')

    plt.subplots_adjust(wspace=-0.01, hspace=-0.01)
    
    
#--------------    
# Tensor utils 
#--------------

def extract_into_tensor(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

#---------------
# Dataset utils
#---------------

class TextImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, subset_name, transform=None, max_cnt=None):
        """
        Arguments:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.extensions = (
            ".jpg",
            ".jpeg",
            ".png",
            ".ppm",
            ".bmp",
            ".pgm",
            ".tif",
            ".tiff",
            ".webp",
        )
        sample_dir = os.path.join(root_dir, subset_name)

        # Collect sample paths
        self.samples = sorted(
            [
                os.path.join(sample_dir, fname)
                for fname in os.listdir(sample_dir)
                if fname[-4:] in self.extensions
            ],
            key=lambda x: x.split("/")[-1].split(".")[0],
        )
        self.samples = (
            self.samples if max_cnt is None else self.samples[:max_cnt]
        )  # restrict num samples

        # Collect captions
        self.captions = {}
        with open(
            os.path.join(root_dir, f"{subset_name}.csv"), newline="\n"
        ) as csvfile:
            spamreader = csv.reader(csvfile, delimiter=",")
            for i, row in enumerate(spamreader):
                if i == 0:
                    continue
                self.captions[row[1]] = row[2]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample_path = self.samples[idx]
        sample = Image.open(sample_path).convert("RGB")

        if self.transform:
            sample = self.transform(sample)

        return {
            "image": sample,
            "text": self.captions[os.path.basename(sample_path)],
            "idxs": idx
        }

## Базовая модель (SD1.5)

Для начала загрузим модель [StableDiffusion 1.5](https://huggingface.co/sd-legacy/stable-diffusion-v1-5) и сгенерируем ей картинки за 5, 10 и 20 шагов.

**Важно:** для экономии памяти, загружаем все компоненты модели в FP16. Не забываем положить модель на GPU.

In [ ]:
pipe = <YOUR CODE HERE>

# Проверяем, что все компоненты модели в FP16 и на cuda
assert pipe.unet.dtype == torch.float16 and pipe.unet.device.type == 'cuda'
assert pipe.vae.dtype == torch.float16 and pipe.vae.device.type == 'cuda'
assert pipe.text_encoder.dtype == torch.float16 and pipe.text_encoder.device.type == 'cuda'

# Заменяем дефолтный сэмплер на DDIM
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")
pipe.scheduler.timesteps = pipe.scheduler.timesteps.cuda()
pipe.scheduler.alphas_cumprod = pipe.scheduler.alphas_cumprod.cuda()

Вызовите pipe для данного промпта, передав ему число шагов, гайденс скейл и генератор случайных чисел. На промпт генерируем 4 картинки.

In [ ]:
prompt = "A sad puppy with large eyes"
guidance_scale = 7.5

for num_inference_steps in [5, 10, 20]:
    generator = torch.Generator('cuda').manual_seed(0)
    
    images = <YOUR CODE HERE>
    
    visualize_images(images)

## Подготовка датасета

Мы будем дообучать модель на небольшой обучающей выборке из 10000 пар текст-картинка сгенерированные моделью [FLUX](https://huggingface.co/black-forest-labs/FLUX.1-dev).

Данные можно загрузить с помощью команд в ячейке ниже. В текущей директории ./ должны появиться:
* Папка flux_data с 10000 картинками
* Файл flux_data.csv с 10000 промптами

Данные парсятся корректным образом в уже реализованном классе **TextImageDataset**.

In [ ]:
!wget https://storage.yandexcloud.net/yandex-research/flux_data_10k.tar.gz
!tar -xzf flux_data_10k.tar.gz

In [ ]:
from torchvision import transforms

transform = transforms.Compose(
    [
        transforms.Resize(512),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
        lambda x: 2 * x - 1,
    ]
)
dataset = TextImageDataset(".",
    subset_name="flux_data",
    transform=transform,
    max_cnt=5000, # Можно варьировать, если очень долго учится
)
assert len(dataset) == 5000 #10000

batch_size = 8 # Рекоммендуемый размер батча на Colab

train_dataloader = torch.utils.data.DataLoader(
    dataset=dataset, shuffle=True, batch_size=batch_size, drop_last=True
)

## Дообучение SD1.5 в Rectified Flow режиме

Функции ниже используются для извлечения эмбедингов из текстовых промптов и перевода картинок в латентное пространство VAE. 

### Задание 1 (0.5 балла)

* Предпосчитать эмбеддинги для пустого промпта "". Будет использоваться для обучения с CFG.
* В функции **prepare_batch** вам нужно замиксовать **uncond_prompt_embeds** в батч с заданной вероятностью

In [ ]:
# Извлекаем эмбеддинги для пустого текста для CFG обучения
# Можно подглядеть в ячейку ниже

uncond_prompt_embeds = <YOUR CORE HERE>

In [ ]:
@torch.no_grad()
def prepare_batch(batch, pipe, uncond_prob=0.1):
    """
    Предобработка батча картинок и текстовых промптов.
    Маппим картинки в латентное пространство VAE.
    Извлекаем эмбеды промптов с помощью текстового энкодера.
    
    Params:

    Return:
        latents: torch.Tensor([B, 4, 64, 64], dtype=torch.float16)
        prompt_embeds: torch.Tensor([B, 77, D], dtype=torch.float16)
    """

    # Токенизируем промпты
    text_inputs = pipe.tokenizer(
        batch['text'],
        padding="max_length",
        max_length=pipe.tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt",
    )

    # Извлекаем эмбеды промптов с помощью текстового энкодера
    prompt_embeds = pipe.text_encoder(text_inputs.input_ids.cuda())[0]
    
    # Замешиваем в батч uncond_prompt_embeds c вероятностью uncond_prob
    <YOUR_CODE_HERE>
    
    # Переводим картинки в латентное пространство VAE
    image = batch['image'].to("cuda", dtype=torch.float16)
    latents = pipe.vae.encode(image).latent_dist.sample()
    latents = latents * pipe.vae.config.scaling_factor
    return latents, prompt_embeds

## Подготовка модели

Для экономии памяти во время обучения будем учить не параметры самой модели, а добавим в нее обучаемые LoRA адаптеры с малым числом параметров.

LoRA представляет собой маленькую добавку к весам модели, где на одну матрицу весов $W \in \mathbb{R}^{m{\times}n} $ обучаются две низкоранговые матрицы $W_A \in \mathbb{R}^{k{\times}n}$ и $W_B \in \mathbb{R}^{k{\times}m}$, где $k$ - ранг матрицы сильно меньше $m$ и $n$.

Тем самым, новая обученная матрица весов может быть представлена как $\hat{W} = W + \Delta W = W + W^T_B W_A$.  
Во время инференса $\Delta W$ можно вмержить в $W$ и получить итоговую модель. 
Также частая практика оставлять адаптеры как есть, чтобы была возможность для одной базовой модели учить несколько адаптеров под разные задачи и переключаться между ними по необходимости.

Если не мержить адаптеры, то вычисления для линейного слоя происходят как на картинке ниже.

<img src=https://storage.yandexcloud.net/yandex-research/cvweek-cd-task-images/lora-idea.jpg width=300>

In [ ]:
# Указываем к каким слоям модели мы будет добавлять адаптеры. Мы здесь указали типичный набор
lora_modules = [
    "to_q", "to_k", "to_v", "to_out.0", "proj_in", "proj_out",
    "ff.net.0.proj", "ff.net.2", "conv1", "conv2", "conv_shortcut",
    "downsamplers.0.conv", "upsamplers.0.conv", "time_emb_proj"
]
lora_config = LoraConfig(
    r=64, # задает ранг у матриц A и B в LoRA.
    lora_alpha=16, # контролирует LR с которым учим LoRA: lr_lora = lr_base * lora_alpha / r. Если учится только LoRA, то можно просто менять LR в оптимизаторе
    target_modules=lora_modules
)

unet = pipe.unet

# Создаем обертку исходной UNet модели с LoRA адаптерами, используя библиотеку PEFT
unet = get_peft_model(unet, lora_config, adapter_name="rf")

# Включаем gradient checkpointing - важная техника для экономии памяти во время обучения
unet.enable_gradient_checkpointing()

# Создаем оптимизатор
optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-4)

## Цикл обучения

### Задание 2 (1.5 балла)

Ниже приведена почти готовая функция обучения модели. **Вам нужно реализовать**: 
* Подготовку входов для модели
* Подсчет таргета и лосса
* Реализация mixed-preciison обучения
* Реалиpовать logit_normal сэмплирование шагов во время обучения. См. секцию 3.1 статье [SD3](https://arxiv.org/pdf/2403.03206) rf/lognorm(0, 1).

Идея за logit_normal сэмплированием - чаще генерировать из середины интервала, где происходят наиболее сложно выучивываемые и значимые шаги. 
В статье [SD3](https://arxiv.org/pdf/2403.03206) сеттинг обучения rf/lognormal(0, 1) показывает лучшие результаты.

### Эффективное обучение
Данное задание рассчитано на успешное выполнение на colab с бесплатной Tesla T4 c 15GB VRAM.
Однако учить даже относительно небольшие T2I модели масштаба SD1.5 на коллабе в лоб проблематично.

Для этого полезно применить ряд инженерных техник, чтобы уместиться в данный бюджет и учиться за разумное время.

**Список техник**

1) Включить **gradient checkpointing** для обучемой модели
2) Добавить **LoRA** (Low Rank Adapters) адаптеры, чтобы учить не все веса, а только 10% добавочных весов
3) Использовать **gradient accumulation**, чтобы делать итерацию обучения по бОльшему батчу, чем влезает по памяти
4) Добавить **mixed precision** FP16/FP32 обучение модели для скорости. Обычно еще и память экономится, но в случае LoRA обучения + gradient checkpointing на память сильно влиять не должно, но зато станет быстрее.
5) **Мульти-GPU** обучение - распределение вычислений по нескольким GPU.  

**Что имеем на данный момент?**

1-2) Уже сделано выше

3 ) Можно реализовать, если по какой-то причине не влезаете по памяти, но поидее и без него все ок

4 ) **Крайне полезно добавить в контексте скорости обучения** 

5 ) Недоступно, так как работаем на одной карточке

### Mixed-precision обучение

Про реализацию mixed-precision в pytorch можно перейти по ссылке: [Mixed-precision обучение](https://pytorch.org/docs/stable/notes/amp_examples.html#typical-mixed-precision-training).

**Учтите**, что нас интересует именно FP16, а не BF16, так как последний не поддерживается на T4 карточках и профита не будет.

**Совет** эта ячейка будет дублироваться в следующем ДЗ, так что в целях экономии времени лучше разобраться с mixed-precision пораньше, учитывая что реализация супер простая.   

In [ ]:
def train_loop(
    model,
    pipe,
    train_dataloader,
    optimizer,
    weighting_scheme='uniform',
    device='cuda'
):    
    # Очищаем память GPU.
    torch.cuda.empty_cache()

    # Инициализация mixed precision.
    scaler = <YOUR CODE HERE>
    
    # Определяем интервал сигм для обучения
    sigmas = torch.linspace(0, 0.999, 1000, device=device)
    
    for i, batch in enumerate(tqdm(train_dataloader)):
        latents, prompt_embeds = prepare_batch(batch, pipe)
        bsz = len(latents)
        
        if weighting_scheme == "logit_normal":
            <YOUR CODE HERE>
        else:
            u = torch.rand(size=(bsz,), device=device)

        # Конвертируем сигмы в шаги, которые будем подавать в модель
        t = <YOUR CODE HERE>
        noise = torch.randn_like(latents)
        sigmas_t = extract_into_tensor(sigmas.cuda(), t, noise.shape)
        
        # Подготовте вход xt
        xt = <YOUR CODE HERE>
        
        # with <YOUR CODE HERE>: # для реализации mixed-precision обучения
        with <YOUR CODE HERE>:
            pred_v = model(
                xt,
                encoder_hidden_states=prompt_embeds,
                timestep=t,
                return_dict=False,
            )[0]
        
        target_v = <YOUR CODE HERE>
        loss = <YOUR CODE HERE>
        
        <YOUR CODE HERE> # Заменить подсчет градиентов и шаг оптимизатора с GradScaler-ом
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        # Печать лосса для мониторинга.
        print(f"Loss: {loss.detach().item():.6f}")

    # Очищаем память GPU.
    torch.cuda.empty_cache()

In [ ]:
train_loop(
    unet,
    pipe,
    train_dataloader,
    optimizer,
)

### Генерируем примеры с помощью нашей модели

Подставляем функции **step_fn** и **schedule_fn** из **Части 1**. Можете использовать любой солвер и расписание на ваш вкус.

In [ ]:
from ysda_hw1.utils.sd15_inference import run

def step_fn(latents, sigmas, v_pred, i, type_, v_pred_fn, cache=None):
    pass


def schedule_fn(n_points, type_, s=3):
    pass

In [ ]:
prompt = "A sad puppy with large eyes"
guidance_scale = 4.5

solver_type = 'dpm2_multi'
schedule_type = 'sd3'

for num_inference_steps in [5, 10, 20]:
    generator = torch.Generator('cuda').manual_seed(0)
    
    sigmas = schedule_fn(
        n_points=num_inference_steps, 
        type_=schedule_type, 
        s=3
    )
    
    images = run(
        pipe,
        prompt=prompt,
        step_fn=functools.partial(step_fn, type_=solver_type),
        sigmas=sigmas,
        num_images_per_prompt=4,
        guidance_scale=guidance_scale,
        generator=generator,
    )

    visualize_images(images)

### Примеры промптов на поиграться


    "portrait photo of a girl, photograph, highly detailed face, depth of field, moody light, golden hour, style by Dan Winters, Steve McCurry, centered, extremely detailed, Nikon D850, award winning photography"
    
    "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"
    
    'A girl with pale blue hair and a cami tank top'
    
    "Four cows in a pen on a sunny day"
    
    "Three dogs sleeping together on an unmade bed"
    
    "A sky blue colored hippopotamus"
    
    "The interior of a mad scientists laboratory, Cluttered with science experiments, tools and strange machine, Eerie purple light, Close up, by Miyazaki"
    
    "a barred owl peeking out from dense tree branches"
    
    "a close-up of a blue dragonfly on a daffodil"
    
    "A green train is coming down the tracks"
    "A photograph of the inside of a subway train. There are frogs sitting on the seats. One of them is reading a newspaper. The window shows the river in the background."
    
    "a family of four posing at the Grand Canyon"
    
    "A high resolution photo of a donkey in a clown costume giving a lecture at the front of a lecture hall. The blackboard has mathematical equations on it. There are many students in the lecture hall."
    
    "A castle made of tortilla chips, in a river made of salsa. There are tiny burritos walking around the castle"
    
    "A raccoon wearing formal clothes, wearing a tophat and holding a cane. The raccoon is holding a garbage bag. Oil painting in the style of abstract cubism."
    
    "A castle made of cardboard."
